In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Using XGBoost Algoritham**

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV

**Remove all Warning messages**

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
customer_df = pd.read_csv("/content/drive/MyDrive/task_dataset_final/custom.csv",sep= ";",decimal= ",")

In [5]:
customer_df

,account length,location code,user id,credit card info save,push status,add to wishlist,desktop sessions,app sessions,desktop transactions,total product detail views,session duration,promotion clicks,avg order value,sale product views,discount rate per visited products,product detail view per app session,app transactions,add to cart per session,customer service calls,churn
0,128,415,3824657,no,yes,25,265,45,17,110,197,87,244.7,91,11.01,10.0,3,2.70,1,0
1,107,415,3717191,no,yes,26,162,27,17,123,196,103,254.4,103,11.45,13.7,3,3.70,1,0
2,137,415,3581921,no,no,0,243,41,10,114,121,110,162.6,104,7.32,12.2,5,3.29,0,0
3,84,408,3759999,yes,no,0,299,51,5,71,62,88,196.9,89,8.86,6.6,7,1.78,2,0
4,75,415,3306626,yes,no,0,167,28,13,113,148,122,186.9,121,8.41,10.1,3,2.73,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,415,4144276,no,yes,36,156,27,18,77,216,126,279.1,83,12.56,9.9,6,2.67,2,0
3329,68,415,3703271,no,no,0,231,39,13,57,153,55,191.3,123,8.61,9.6,4,2.59,3,0
3330,28,510,3288230,no,no,0,181,31,25,109,289,58,191.9,91,8.64,14.1,6,3.81,2,0
3331,184,510,3646381,yes,no,0,214,36,14,105,160,84,139.2,137,6.26,5.0,10,1.35,2,0


In [6]:
customer_df.shape

(3333, 20)

In [7]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   account length                       3333 non-null   int64  
 1   location code                        3333 non-null   int64  
 2   user id                              3333 non-null   int64  
 3   credit card info save                3333 non-null   object 
 4   push status                          3333 non-null   object 
 5   add to wishlist                      3333 non-null   int64  
 6   desktop sessions                     3333 non-null   int64  
 7   app sessions                         3333 non-null   int64  
 8   desktop transactions                 3333 non-null   int64  
 9   total product detail views           3333 non-null   int64  
 10  session duration                     3333 non-null   int64  
 11  promotion clicks              

In [8]:
customer_df["location code"].unique()

array([415, 408, 510])

In [9]:
customer_df["location code"] = customer_df["location code"].astype(str)

In [10]:
customer_df["credit card info save"].unique()

array(['no', 'yes'], dtype=object)

In [11]:
customer_df["credit card info save"] = customer_df["credit card info save"].replace({"yes": 1, "no": 0})
customer_df["push status"] = customer_df["push status"].replace({"yes": 1, "no": 0})

In [12]:
customer_df

,account length,location code,user id,credit card info save,push status,add to wishlist,desktop sessions,app sessions,desktop transactions,total product detail views,session duration,promotion clicks,avg order value,sale product views,discount rate per visited products,product detail view per app session,app transactions,add to cart per session,customer service calls,churn
0,128,415,3824657,0,1,25,265,45,17,110,197,87,244.7,91,11.01,10.0,3,2.70,1,0
1,107,415,3717191,0,1,26,162,27,17,123,196,103,254.4,103,11.45,13.7,3,3.70,1,0
2,137,415,3581921,0,0,0,243,41,10,114,121,110,162.6,104,7.32,12.2,5,3.29,0,0
3,84,408,3759999,1,0,0,299,51,5,71,62,88,196.9,89,8.86,6.6,7,1.78,2,0
4,75,415,3306626,1,0,0,167,28,13,113,148,122,186.9,121,8.41,10.1,3,2.73,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,415,4144276,0,1,36,156,27,18,77,216,126,279.1,83,12.56,9.9,6,2.67,2,0
3329,68,415,3703271,0,0,0,231,39,13,57,153,55,191.3,123,8.61,9.6,4,2.59,3,0
3330,28,510,3288230,0,0,0,181,31,25,109,289,58,191.9,91,8.64,14.1,6,3.81,2,0
3331,184,510,3646381,1,0,0,214,36,14,105,160,84,139.2,137,6.26,5.0,10,1.35,2,0


In [13]:
customer_df["add to cart per session"] = customer_df["add to cart per session"].astype(float)


In [14]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   account length                       3333 non-null   int64  
 1   location code                        3333 non-null   object 
 2   user id                              3333 non-null   int64  
 3   credit card info save                3333 non-null   int64  
 4   push status                          3333 non-null   int64  
 5   add to wishlist                      3333 non-null   int64  
 6   desktop sessions                     3333 non-null   int64  
 7   app sessions                         3333 non-null   int64  
 8   desktop transactions                 3333 non-null   int64  
 9   total product detail views           3333 non-null   int64  
 10  session duration                     3333 non-null   int64  
 11  promotion clicks              

In [15]:
customer_df = pd.get_dummies(customer_df, columns = ["location code"])

In [16]:
customer_df

,account length,user id,credit card info save,push status,add to wishlist,desktop sessions,app sessions,desktop transactions,total product detail views,session duration,...,sale product views,discount rate per visited products,product detail view per app session,app transactions,add to cart per session,customer service calls,churn,location code_408,location code_415,location code_510
0,128,3824657,0,1,25,265,45,17,110,197,...,91,11.01,10.0,3,2.70,1,0,0,1,0
1,107,3717191,0,1,26,162,27,17,123,196,...,103,11.45,13.7,3,3.70,1,0,0,1,0
2,137,3581921,0,0,0,243,41,10,114,121,...,104,7.32,12.2,5,3.29,0,0,0,1,0
3,84,3759999,1,0,0,299,51,5,71,62,...,89,8.86,6.6,7,1.78,2,0,1,0,0
4,75,3306626,1,0,0,167,28,13,113,148,...,121,8.41,10.1,3,2.73,3,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,4144276,0,1,36,156,27,18,77,216,...,83,12.56,9.9,6,2.67,2,0,0,1,0
3329,68,3703271,0,0,0,231,39,13,57,153,...,123,8.61,9.6,4,2.59,3,0,0,1,0
3330,28,3288230,0,0,0,181,31,25,109,289,...,91,8.64,14.1,6,3.81,2,0,0,0,1
3331,184,3646381,1,0,0,214,36,14,105,160,...,137,6.26,5.0,10,1.35,2,0,0,0,1


In [17]:
customer_df = customer_df.drop("user id", axis = 1)

**Scalling big numbers between 0 to 1**

In [18]:
customer_df.columns

Index(['account length', 'credit card info save', 'push status',
       'add to wishlist', 'desktop sessions', 'app sessions',
       'desktop transactions', 'total product detail views',
       'session duration', 'promotion clicks', 'avg order value',
       'sale product views', 'discount rate per visited products',
       'product detail view per app session', 'app transactions',
       'add to cart per session', 'customer service calls', 'churn',
       'location code_408', 'location code_415', 'location code_510'],
      dtype='object')

In [19]:
cols_to_scale = ['account length',
       'add to wishlist', 'desktop sessions', 'app sessions',
       'desktop transactions', 'total product detail views',
       'session duration', 'promotion clicks', 'avg order value',
       'sale product views', 'discount rate per visited products',
       'product detail view per app session', 'app transactions',
       'add to cart per session', 'customer service calls']

In [20]:
scaler = Normalizer()
scaled_data = scaler.fit_transform(customer_df[cols_to_scale])
scaled_df = pd.DataFrame(scaled_data, index = customer_df.index, columns = cols_to_scale)

In [21]:
scaled_df

,account length,add to wishlist,desktop sessions,app sessions,desktop transactions,total product detail views,session duration,promotion clicks,avg order value,sale product views,discount rate per visited products,product detail view per app session,app transactions,add to cart per session,customer service calls
0,0.275142,0.053739,0.569631,0.096730,0.036542,0.236451,0.423461,0.187011,0.525995,0.195609,0.023667,0.021496,0.006449,0.005804,0.002150
1,0.252755,0.061417,0.382676,0.063779,0.040157,0.290551,0.462991,0.243307,0.600944,0.243307,0.027047,0.032362,0.007087,0.008740,0.002362
2,0.345945,0.000000,0.613611,0.103531,0.025251,0.287867,0.305543,0.277766,0.410589,0.262615,0.018484,0.030807,0.012626,0.008308,0.000000
3,0.208327,0.000000,0.741543,0.126484,0.012400,0.176086,0.153765,0.218247,0.488327,0.220727,0.021973,0.016369,0.017361,0.004415,0.004960
4,0.205041,0.000000,0.456559,0.076549,0.035541,0.308929,0.404615,0.333534,0.510963,0.330800,0.022992,0.027612,0.008202,0.007464,0.008202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,0.412072,0.077263,0.334808,0.057948,0.038632,0.165258,0.463580,0.270422,0.599006,0.178135,0.026956,0.021247,0.012877,0.005730,0.004292
3329,0.180912,0.000000,0.614569,0.103758,0.034586,0.151647,0.407052,0.146326,0.508948,0.327238,0.022907,0.025541,0.010642,0.006891,0.007981
3330,0.066120,0.000000,0.427415,0.073204,0.059035,0.257394,0.682448,0.136962,0.453155,0.214888,0.020403,0.033296,0.014168,0.008997,0.004723
3331,0.455551,0.000000,0.529825,0.089129,0.034661,0.259961,0.396131,0.207969,0.344634,0.339187,0.015499,0.012379,0.024758,0.003342,0.004952


In [22]:
customer_df = customer_df.drop(cols_to_scale, axis = 1)
customer_df = pd.merge(customer_df, scaled_df, left_index=True, right_index=True)

**Split data into test and train**

In [23]:
customer_df

,credit card info save,push status,churn,location code_408,location code_415,location code_510,account length,add to wishlist,desktop sessions,app sessions,...,total product detail views,session duration,promotion clicks,avg order value,sale product views,discount rate per visited products,product detail view per app session,app transactions,add to cart per session,customer service calls
0,0,1,0,0,1,0,0.275142,0.053739,0.569631,0.096730,...,0.236451,0.423461,0.187011,0.525995,0.195609,0.023667,0.021496,0.006449,0.005804,0.002150
1,0,1,0,0,1,0,0.252755,0.061417,0.382676,0.063779,...,0.290551,0.462991,0.243307,0.600944,0.243307,0.027047,0.032362,0.007087,0.008740,0.002362
2,0,0,0,0,1,0,0.345945,0.000000,0.613611,0.103531,...,0.287867,0.305543,0.277766,0.410589,0.262615,0.018484,0.030807,0.012626,0.008308,0.000000
3,1,0,0,1,0,0,0.208327,0.000000,0.741543,0.126484,...,0.176086,0.153765,0.218247,0.488327,0.220727,0.021973,0.016369,0.017361,0.004415,0.004960
4,1,0,0,0,1,0,0.205041,0.000000,0.456559,0.076549,...,0.308929,0.404615,0.333534,0.510963,0.330800,0.022992,0.027612,0.008202,0.007464,0.008202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,0,1,0,0,1,0,0.412072,0.077263,0.334808,0.057948,...,0.165258,0.463580,0.270422,0.599006,0.178135,0.026956,0.021247,0.012877,0.005730,0.004292
3329,0,0,0,0,1,0,0.180912,0.000000,0.614569,0.103758,...,0.151647,0.407052,0.146326,0.508948,0.327238,0.022907,0.025541,0.010642,0.006891,0.007981
3330,0,0,0,0,0,1,0.066120,0.000000,0.427415,0.073204,...,0.257394,0.682448,0.136962,0.453155,0.214888,0.020403,0.033296,0.014168,0.008997,0.004723
3331,1,0,0,0,0,1,0.455551,0.000000,0.529825,0.089129,...,0.259961,0.396131,0.207969,0.344634,0.339187,0.015499,0.012379,0.024758,0.003342,0.004952


In [24]:
x = customer_df.drop("churn", axis=1)
y = customer_df["churn"]

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [26]:
y_train.shape

(2666,)

In [27]:
x_test.shape

(667, 20)

Build the Model

In [28]:
xgb_cl = xgb.XGBClassifier()
xgb_cl.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [29]:
preds = xgb_cl.predict(x_test)

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
acc = accuracy_score(y_test, preds)
print(acc)

0.9280359820089955


**Hyperparameter Tuning**

In [32]:
param_grid = {
    "max_depth": [5],
    "learning_rate": [0, 0.01, 0.05, 0.1],
    "gamma": [1, 5, 10],
    "scale_pos_weight": [2, 5,10, 20],
    "subsample": [1],
    "colsample_bytree": [1]
}

xgb_cl2 = xgb.XGBClassifier(objective="binary:logistic")
grid_cv = GridSearchCV(xgb_cl2, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")
_ = grid_cv.fit(x_train, y_train)
print(grid_cv.best_score_)
print(grid_cv.best_params_)


0.8858997706269248
{'colsample_bytree': 1, 'gamma': 1, 'learning_rate': 0.05, 'max_depth': 5, 'scale_pos_weight': 2, 'subsample': 1}


In [34]:
final_cl = xgb.XGBClassifier(
    **grid_cv.best_params_, objective="binary:logistic"
)
grid_final = final_cl.fit(x_train, y_train)
preds = grid_final.predict(x_test)
acc = accuracy_score(y_test, preds)
print("Accuracy of the Final Model:", acc)

Accuracy of the Final Model: 0.9280359820089955
